In [1]:
import json

In [2]:
changes = json.load(open("diffs_by_variable.json"))

In [3]:
labels = json.load(open('labels.json'))

In [4]:
cmip7_to_6_cn = {}

for k,v in labels.items():
    _,cn,_ = v['title'].split()
    elements = cn.split('.')
    region = elements[-1]
    if region != "30S-90S":
        elements[-1] = region.lower()
        cn = '.'.join(elements)
    cmip7_to_6_cn[cn] = k


In [5]:
cmip7_to_6_cn

{'aerosol.rsdcsaf.tavg-u-hxy-u.mon.glb': 'AERmon.rsdcsaf',
 'aerosol.rsdaf.tavg-u-hxy-u.mon.glb': 'AERmon.rsdaf',
 'aerosol.rldcsaf.tavg-u-hxy-u.mon.glb': 'AERmon.rldcsaf',
 'aerosol.rldaf.tavg-u-hxy-u.mon.glb': 'AERmon.rldaf',
 'ocean.tos.tavg-u-hm-sea.mon.glb': 'Omon.tosga',
 'ocnBgchem.dpco2.tavg-u-hxy-sea.mon.glb': 'Omon.dpco2',
 'landIce.snc.tavg-u-hxy-lnd.mon.glb': 'LImon.snc',
 'atmos.tntd.tavg-al-hxy-u.mon.glb': 'Emon.tntd',
 'land.raLut.tavg-u-hxy-multi.mon.glb': 'Emon.raLut',
 'land.nLitterSurf.tavg-u-hxy-lnd.mon.glb': 'Emon.nLitterSurf',
 'atmos.vtem.tavg-p39-hy-air.day.glb': 'EdayZ.vtem',
 'atmos.zfull.ti-al-hxy-u.fx.glb': 'fx.zfull',
 'land.sftlaf.ti-u-hxy-u.fx.glb': 'fx.sftlaf',
 'atmos.sftlf.ti-u-hxy-u.fx.glb': 'fx.sftlf',
 'land.sftgif.ti-u-hxy-u.fx.glb': 'fx.sftgif',
 'land.rootd.ti-u-hxy-lnd.fx.glb': 'fx.rootd',
 'land.orog.ti-u-hxy-u.fx.glb': 'fx.orog',
 'land.mrsofc.ti-u-hxy-lnd.fx.glb': 'fx.mrsofc',
 'atmos.lon.ti-u-hs-u.fx.glb': 'fx.lon',
 'landIce.lithk.ti-u-hxy-

In [6]:
len(cmip7_to_6_cn)

1834

In [7]:
from importlib import reload
import dr_issue
reload(dr_issue)

<module 'dr_issue' from '/home/users/matthew.mizielinski/UKNCSP/CDDS-CMIP7-mappings/construction/update_1.2.2.3/dr_issue.py'>

In [8]:
field_mapping = {
    'long_name' : 'Long name',
    'cell_measures': 'Cell measures',
    'cell_methods': 'Cell methods',
    'comment': 'Comment',
    'modeling_realm': 'Modeling realm',
    'standard_name': 'CF standard name'
}

allowed_mapping_keys = [
    'Expression HadGEM3-GC31',
    "Expression HadGEM3-GC5" ,
    "Expression UKESM1",
    "Expression UKESM1-3",
    'Expression UKESM2',
    "Model units"]

mapping_key_changes = {
    'Expression HadGEM3-GC3.1':'Expression HadGEM3-GC31',
    'Expression HadGEM-GC3.1': 'Expression HadGEM3-GC31',
    'Expression HadGEm3-GC5': 'Expression HadGEM3-GC5',
    'Expression UKESM1.3': 'Expression UKESM1-3',
    '** Model units': 'Model units',
    'Expression UKESM 1-3': 'Expression UKESM1-3',
    'Expression HadGEM-GC5': 'Expression HadGEM3-GC5',
    'Expression HadGEM3-GC3-1': 'Expression HadGEM3-GC31',
}

allowed_stash_keys = [
    'HadGEM3-GC31',
    "HadGEM3-GC5" ,
    "UKESM1",
    "UKESM1-3",
    'UKESM2',]

stash_key_changes = {
    'HadGEM3-GC3.1': 'HadGEM3-GC31',
    "UKESM1.3": "UKESM1-3",
    'HadGEM3-GC3-1': 'HadGEM3-GC31',
    'HadgEM3-GC5': 'HadGEM3-GC5',
}

for c7_cn, c6_cn in cmip7_to_6_cn.items():
    issue_changed = False
    filename = 'issues/' + c6_cn
    issue = dr_issue.DRIssue()
    issue.read_file('issues/'+c6_cn)

    if 'Region' in issue.mapping_info:
        raise RuntimeError('here')
    
    region = issue.dr_info['Region']
    if region != '30S-90S' and region != region.lower():
        issue.dr_info['Region'] = region.lower()
        if issue.dr_notes['Region']:
            issue.dr_notes['Region'] += "; "
        issue.dr_notes['Region'] += "Updated DR v1.2.2.3 (lower case)"
        issue_changed = True

    if c7_cn in changes['Compound Name']:
        differences = changes['Compound Name'][c7_cn]
        for field, update in differences.items():
            dr_field = field_mapping[field]
            if dr_field not in issue.dr_info:
                raise KeyError()
            issue.dr_info[dr_field] = update['v1223.json']
            if issue.dr_notes[dr_field]:
                issue.dr_notes[dr_field] += "; "
            issue.dr_notes[dr_field] += "Updated DR v1.2.2.3"
        issue_changed = True

    for k in list(issue.mapping_info.keys()):
        if k and k not in allowed_mapping_keys:
            if k in mapping_key_changes:
                nk = mapping_key_changes[k]
                issue.mapping_info[nk] = issue.mapping_info[k]
                issue.mapping_notes[nk] = issue.mapping_notes[k]
                del issue.mapping_info[k]
                del issue.mapping_notes[k]
            else:
                raise RuntimeError(repr(k))

    for k in list(issue.stash.keys()):
         if k and k not in allowed_stash_keys:
             if k in stash_key_changes:
                 nk = stash_key_changes[k]
                 issue.stash[nk] = issue.stash[k]
                 del issue.stash[k]
             else:
                 raise RuntimeError(repr(k))
    
    
    if issue_changed:
        print(c6_cn)
        issue.write_file(filename)
    

AERmon.rsdcsaf
AERmon.rsdaf
AERmon.rldcsaf
AERmon.rldaf
Omon.tosga
Omon.dpco2
LImon.snc
Emon.tntd
Emon.raLut
Emon.nLitterSurf
EdayZ.vtem
fx.zfull
fx.sftlaf
fx.sftlf
fx.sftgif
fx.rootd
fx.orog
fx.mrsofc
fx.lon
fx.lithk
fx.lat
fx.depthsl
fx.depthl
fx.areacellr
fx.areacella
day.zg
day.wbgt2mmax
day.wbgt2m
day.wap
day.vas
day.va
day.uas
day.ua
day.tslsi
day.tasmin
day.tasmax
day.tas
day.ta
day.snw
day.snc
day.sfcWindmax
day.sfcWind
day.rsus
day.rss
day.rsds
day.rlut
day.rluscs
day.rlus
day.rls
day.rlds
day.psl
day.prsn
day.prc
day.pr
day.noaahi2mmax
day.noaahi2m
day.mrsos
day.mrso
day.mrro
day.irrSurf
day.irrLut
day.irrGw
day.irrDem
day.huss
day.hus
day.hursmin
day.hursmax
day.hurs
day.hur
day.hfss
day.hfls
day.clt
SImon.sivols
SImon.sivoln
SImon.sivol
SImon.siv
SImon.siu
SImon.sitimefrac
SImon.sithick
SImon.sitemptop
SImon.sitempsnic
SImon.sitempbot
SImon.sistryubot
SImon.sistrydtop
SImon.sistrxubot
SImon.sistrxdtop
SImon.sistressmax
SImon.sistressave
SImon.sispeed
SImon.sisnthick
SImon.s

In [9]:
print(issue.write())



## Data Request information

| Field | Value | Notes |
| --- | --- | --- |
| Key | Amon.aod550volso4 | |
| Branded variable name | aod550volso4_tavg-u-hxy-u | |
| CF standard name | stratosphere_optical_thickness_due_to_volcanic_ambient_aerosol_particles | |
| Long name | Monthly Aerosol Optical Depth at 550nm Due to Stratospheric Volcanic Aerosols | |
| Comment | This is the monthly averaged aerosol optical thickness at 550 nm due to stratospheric volcanic sulphate aerosols | |
| Dimensions | longitude latitude time lambda550nm | |
| Frequency | mon | |
| Units | 1E-09 | |
| Positive | | |
| Processing notes | | |
| Cell methods | area: time: mean | |
| Cell measures | area: areacella | |
| Modeling realm | atmos | |
| Variable status | New | |
| CMIP6 Differences | Not present | |
| Region | glb | Updated DR v1.2.2.3 (lower case) |


## Mapping information 

| Field | Value | Notes |
| --- | --- | --- |
| Model units | | |


## STASH entries (relevant for UM only)

| Model | STASH |